# Tabular Model: sequential data, with context

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mostly-ai/mostlyai-engine/blob/main/examples/sequential.ipynb)

In [6]:
from pathlib import Path
import pandas as pd
import numpy as np
from mostlyai import engine

# init workspace and logging
ws = Path("ws-tabular-sequential")
engine.init_logging()

# load original data
url = "https://github.com/mostly-ai/public-demo-data/raw/refs/heads/dev/baseball"
trn_ctx_df = pd.read_csv(f"{url}/players.csv.gz")  # context data
trn_tgt_df = pd.read_csv(f"{url}/batting.csv.gz")  # target data



In [7]:
trn_ctx_df.head()

,id,country,birthDate,deathDate,nameFirst,nameLast,weight,height,bats,throws
0,aardsda01,USA,1981-12-27,NaN,David,Aardsma,215.0,75.0,R,R
1,aaronha01,USA,1934-02-05,2021-01-22,Hank,Aaron,180.0,72.0,R,R
2,aaronto01,USA,1939-08-05,1984-08-16,Tommie,Aaron,190.0,75.0,R,R
3,aasedo01,USA,1954-09-08,NaN,Don,Aase,190.0,75.0,R,R
4,abadan01,USA,1972-08-25,NaN,Andy,Abad,184.0,73.0,L,L


In [5]:
# execute the engine steps
from pathlib import Path
import pandas as pd
import numpy as np
from mostlyai import engine

# init workspace and logging
ws = Path("ws-tabular-sequential")
engine.init_logging()

# load original data
url = "https://github.com/mostly-ai/public-demo-data/raw/refs/heads/dev/cdnow"
trn_ctx_df = pd.read_csv(f"{url}/users.csv.gz")  # context data
trn_tgt_df = pd.read_csv(f"{url}/purchases.csv.gz")  # target data

engine.split(                         # split data as PQT files for `trn` + `val` to `{ws}/OriginalData/(tgt|ctx)-data`
  workspace_dir=ws,
  tgt_data=trn_tgt_df,
  ctx_data=trn_ctx_df,
  tgt_context_key="users_id",
  ctx_primary_key="id",
  model_type="TABULAR",
)
engine.analyze(workspace_dir=ws)      # generate column-level statistics to `{ws}/ModelStore/(tgt|ctx)-data/stats.json`
engine.encode(workspace_dir=ws)       # encode training data to `{ws}/OriginalData/encoded-data`
engine.train(                         # train model and store to `{ws}/ModelStore/model-data`
    workspace_dir=ws,
    max_training_time=2,              # limit TRAIN to 2 minute for demo purposes
)
engine.generate(workspace_dir=ws)     # use model to generate synthetic samples to `{ws}/SyntheticData`

[2025-07-09 11:28:30,443] INFO   : SPLIT started
[2025-07-09 11:28:30,447] INFO   : clean `ws-tabular-sequential/OriginalData/tgt-data`
[2025-07-09 11:28:30,448] INFO   : clean `ws-tabular-sequential/OriginalData/tgt-meta`
[2025-07-09 11:28:30,448] INFO   : clean `ws-tabular-sequential/OriginalData/ctx-data`
[2025-07-09 11:28:30,449] INFO   : clean `ws-tabular-sequential/OriginalData/ctx-meta`
[2025-07-09 11:28:30,449] INFO   : model_type='TABULAR'
[2025-07-09 11:28:30,449] INFO   : tgt_encoding_types={'date': 'TABULAR_CATEGORICAL', 'cds': 'TABULAR_NUMERIC_AUTO', 'amt': 'TABULAR_NUMERIC_AUTO'}
[2025-07-09 11:28:30,469] INFO   : SPLIT finished in 0.02s
[2025-07-09 11:28:30,469] INFO   : ANALYZE started
[2025-07-09 11:28:30,470] INFO   : clean `ws-tabular-sequential/ModelStore/tgt-stats`
[2025-07-09 11:28:30,471] INFO   : clean `ws-tabular-sequential/ModelStore/ctx-stats`
[2025-07-09 11:28:30,472] INFO   : analyzing 2 partitions in parallel
[2025-07-09 11:28:31,081] INFO   : analyzed tar

KeyError: 'tgt:/__sidx_cat'

In [ ]:
# load synthetic data
syn_tgt_df = pd.read_parquet(ws / "SyntheticData")
syn_tgt_df.head(5)

### QUALITY ASSURANCE

#### sequence lengths

In [ ]:
trn_seq_lens = trn_tgt_df.groupby("players_id").size()
syn_seq_lens = syn_tgt_df.groupby("players_id").size()

In [ ]:
print("tgt: ", np.quantile(trn_seq_lens, np.arange(0, 1.1, 0.1), method="inverted_cdf"))
print("syn: ", np.quantile(syn_seq_lens, np.arange(0, 1.1, 0.1), method="inverted_cdf"))

#### coherence

In [ ]:
syn_avg_teams_per_player = syn_tgt_df.groupby("players_id")["team"].nunique().mean().round(1)
trn_avg_teams_per_player = trn_tgt_df.groupby("players_id")["team"].nunique().mean().round(1)
syn_avg_teams_per_player, trn_avg_teams_per_player